## Matchmaking Algorithm Classifier
- Bert Transformer / Bloom

Author: Mark Cheung,Claire Song

In [ ]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
!pip install transformers


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.1 MB/s eta 0:00:00


In [ ]:
import time
import torch
import logging
import transformers
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import pdb
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
transformers.__version__

model_choice = 'bloom'
model_choice = 'bert'

In [ ]:
if model_choice == 'bert':
  tokenizer_class = transformers.BertTokenizer
  pretrained_weights='bert-base-uncased'
  tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
else:
  tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom", is_split_into_words= True)

logging.getLogger("transformers").setLevel(logging.ERROR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_path = '/content/drive/My Drive/New Data/'
# df = pd.read_csv(data_path + 'issues_descriptions_06_21.csv')
df = pd.read_csv(data_path + 'new_data_cleaned.csv')
#df = pd.read_csv(data_path + 'new_data_cleaned_removed_keywords_06_29.csv')
issues = df.iloc[:1]
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.sample(frac=1, random_state = 42) # reorder
# print(df.iloc[0])

In [ ]:
number_of_issues = 6
values = df.loc[:,"Description"].to_numpy()
labels = df[df.columns[1:number_of_issues+1]].to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(values, labels, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

df = pd.DataFrame(y_train)
df2 = pd.DataFrame(X_train)
trainset = pd.concat([df2,df],axis=1)

df = pd.DataFrame(y_val)
df2 = pd.DataFrame(X_val)
valset = pd.concat([df2,df],axis=1)

df = pd.DataFrame(y_test)
df2 = pd.DataFrame(X_test)
testset = pd.concat([df2,df],axis=1)
# print(trainset.iloc[0])

In [ ]:
class Dataset(Dataset):
    def __init__(self, dataframe, max_len):
        self.dataframe = dataframe
        self.max_len = max_len
        self.sep_id = tokenizer.encode('[SEP]', add_special_tokens=False)
        self.pad_id = tokenizer.encode('[PAD]', add_special_tokens=False)
        # self.sep_id = tokenizer.encode(['[SEP]'], add_special_tokens=False)
        # self.pad_id = tokenizer.encode(['[PAD]'], add_special_tokens=False)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe[idx]
        text = row[0]
        # targets = torch.tensor(list(row[1:]))
        encoded = tokenizer.encode(text, add_special_tokens=True)[:self.max_len-1] #.tolist()
        if encoded[-1] != self.sep_id[0]:
            encoded = encoded + self.sep_id
        padded = encoded + self.pad_id * (self.max_len - len(encoded))
        padded = torch.tensor(padded)
        labels = torch.Tensor(list(row[1:]))
        return padded, labels

In [ ]:
def evaluate(model, data, threshhold):
    actual, predictions = [], []
    with torch.no_grad():
        for features, targets in data:
            features = features.to(device)
            targets = targets.to(device)
            scores = model(features)
            sigmoid_out = torch.sigmoid(scores)

            prediction = torch.argmax(sigmoid_out,dim =1)
            targets = torch.argmax(targets,dim = 1)
            # prediction = torch.as_tensor(sigmoid_out > threshhold, dtype=torch.int32)
            #prediction = torch.as_tensor(sigmoid_out > 0.5, dtype=torch.int32)
            predictions.extend(prediction.view(-1).tolist())
            actual.extend(targets.long().view(-1).tolist())
    assert len(actual) == len(predictions)

    return f1_score(actual, predictions, average = 'macro')
    #print('Macro F1 Score', f1_score(actual, predictions, average = 'macro'))

In [ ]:
class modelNN(transformers.PreTrainedModel):
    def __init__(self, config, hidden_size,number_of_issues, model='bert' ):
        super(modelNN, self).__init__(config)
        if model =='bert':
          # self.model = transformers.BertModel.from_pretrained( pretrained_weights, output_attentions = True)
          self.model = transformers.BertModel(config)

        elif model =='bloom':
          self.model = BloomForCausalLM.from_pretrained("bigscience/bloom-1b7", output_attentions = True)

        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(hidden_size, number_of_issues)

    def forward(self, ex):
        # _, pooled_output, attentions = self.model(ex, return_dict = False)

        _, pooled_output = self.model(ex, return_dict = False)
        pooled_output = self.dropout(pooled_output)
        fc_out = self.fc(pooled_output)
        # return fc_out, attentions
        return fc_out

In [ ]:
# for label in df.keys()[1:]:
  # print('Current label is {}'.format(label))

train_dataset = Dataset(tuple(trainset.itertuples(index=False, name=None)), 96)
train_dataloader = DataLoader(train_dataset, batch_size=32, 
                              shuffle=True, num_workers=2)

val_dataset = Dataset(tuple(valset.itertuples(index=False, name=None)), 96)
val_dataloader = DataLoader(val_dataset, batch_size=32, 
                            shuffle=True, num_workers=2)

test_dataset = Dataset(tuple(testset.itertuples(index=False, name=None)), 96)
test_dataloader = DataLoader(test_dataset, batch_size=32, 
                              shuffle=True, num_workers=2)

# model setting
from transformers import BertConfig, BertModel
config = BertConfig()
model = modelNN(config, 768, number_of_issues,model_choice)
model = model.to(device)

loss_function = nn.BCEWithLogitsLoss()
loss_function = loss_function.to(device)
optimizer = transformers.AdamW(model.parameters(), lr=2e-5, correct_bias=False)
MAX_EPOCHS = 30 #TODO

max_grad_norm = 1.0
warmup_proportion = 0.1
num_training_steps  = len(train_dataloader) * MAX_EPOCHS
num_warmup_steps = num_training_steps * warmup_proportion
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
      num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# epoch training on test on validation set
for epoch in range(MAX_EPOCHS):
    epoch_loss = 0
    start_time = time.time()
    for idx, (features, targets) in enumerate(train_dataloader):
        model.zero_grad()
        features = features.to(device)
        targets = targets.to(device)
        scores = model(features)
        loss = loss_function(scores, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()
    if device == 'cuda':
        torch.cuda.empty_cache()
    time_taken = round((time.time() - start_time)/60, 2)
    if epoch%10 == 0:
      print('Epoch {} | Loss - {} | Time Taken - {} min'.format(epoch+1,
              epoch_loss,time_taken) )
      print('Macro F1 Score',evaluate(model, val_dataloader, 0.5))

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 | Loss - 46.23268747329712 | Time Taken - 1.01 min
Macro F1 Score 0.09158143400276146
Epoch 11 | Loss - 16.641054935753345 | Time Taken - 0.87 min
Macro F1 Score 0.6014480418354327
Epoch 21 | Loss - 5.7828895933926105 | Time Taken - 0.9 min
Macro F1 Score 0.6166785497066795


In [ ]:
# testing on test set

# calculate precision and recall and F1-score


data = test_dataloader
actual, predictions = [], []
sentences = []
pred = []
gold = []
with torch.no_grad():
    for features, targets in data:
        features = features.to(device)
        
        targets = targets.to(device)
        scores = model(features)
        sigmoid_out = torch.sigmoid(scores)
        # prediction = torch.as_tensor(sigmoid_out > 0.5, dtype=torch.int32)
        prediction = torch.argmax(sigmoid_out,dim =1)
        targets = torch.argmax(targets,dim = 1)

        predictions.extend(prediction.view(-1).tolist())
        actual.extend(targets.long().view(-1).tolist())
print(f1_score(actual,predictions, average='macro'))
print(confusion_matrix(actual,predictions, normalize='true'))


0.6154786408795866
[[0.47317073 0.10731707 0.32682927 0.0097561  0.05853659 0.02439024]
 [0.07692308 0.46153846 0.38461538 0.01183432 0.0591716  0.00591716]
 [0.14619883 0.24561404 0.50877193 0.02923977 0.04093567 0.02923977]
 [0.01092896 0.06557377 0.06010929 0.8579235  0.         0.00546448]
 [0.12       0.095      0.135      0.005      0.64       0.005     ]
 [0.13274336 0.00884956 0.18584071 0.00884956 0.01769912 0.6460177 ]]


In [ ]:
# save / load model
from google.colab import files
import torch 
# torch.save(model.state_dict(), 'model_state_dict.pth')
# files.download('model_state_dict.pth')

model.save_pretrained("saved_model/model_pretrained")
# files.download('model.pretrained')




In [ ]:
# passing description to model

# data = Dataset([["description of issue", 1]],96)
# train_dataloader = DataLoader(data)
# feats, _ = next(iter(train_dataloader))
# scores, _ = model(feats)
# print(torch.sigmoid(scores))